In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.applications import VGG16, VGG19, ResNet101
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

In [2]:
# Set parameters
IMG_SIZE = (128, 128)
BATCH_SIZE = 128
EPOCHS = 5
LEARNING_RATE = 0.0001
RANDOM_STATE = 32
DATA_DIR = "images"

In [3]:
# Create a folder to save models if it does not exist
os.makedirs("models", exist_ok=True)

# Create a folder to save the plots if it does not exist
os.makedirs("plots", exist_ok=True)

In [4]:
# 1. Data Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

# Training data (70%)
train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=RANDOM_STATE
)

# Validation data (20%)
val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=RANDOM_STATE
)

# Testing data (10%)
test_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
    seed=RANDOM_STATE
)

Found 13563 images belonging to 2 classes.
Found 5811 images belonging to 2 classes.
Found 19374 images belonging to 2 classes.


In [5]:
# 2. Define Models
def create_cnn():
    model = Sequential([
        Input(shape=(128, 128, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_vgg16():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_vgg19():
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_resnet101():
    base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

# Compile function
def compile_model(model):
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
# 3. Model Training and Evaluation
models = {
    "CNN": create_cnn(),
    "VGG16": create_vgg16(),
    "VGG19": create_vgg19(),
    "ResNet101": create_resnet101()
}

history_dict = {}
results = {}

for model_name, model in models.items():
    print(f"Training {model_name} model...")
    model = compile_model(model)
    model.summary()
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1
    )
    # Store history
    history_dict[model_name] = history
    # Evaluate model
    loss, accuracy = model.evaluate(test_gen, verbose=0)
    results[model_name] = (loss, accuracy)
    print(f"{model_name} - Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")
    
    # Save the trained model
    model_filename = f"models/{model_name.lower()}_model.h5"
    model.save(model_filename)
    print(f"{model_name} model saved successfully at: {model_filename}")

Training CNN model...
Epoch 1/5
106/106 [==============================] - 503s 5s/step - loss: 0.5720 - accuracy: 0.6900 - val_loss: 0.6041 - val_accuracy: 0.6825
Epoch 2/5
106/106 [==============================] - 404s 4s/step - loss: 0.4820 - accuracy: 0.7697 - val_loss: 0.5232 - val_accuracy: 0.7443
Epoch 3/5
106/106 [==============================] - 439s 4s/step - loss: 0.4432 - accuracy: 0.7943 - val_loss: 0.5259 - val_accuracy: 0.7503
Epoch 4/5
106/106 [==============================] - 413s 4s/step - loss: 0.4159 - accuracy: 0.8099 - val_loss: 0.4978 - val_accuracy: 0.7551
Epoch 5/5
106/106 [==============================] - 465s 4s/step - loss: 0.3930 - accuracy: 0.8270 - val_loss: 0.4848 - val_accuracy: 0.7689
CNN - Test Loss: 0.3981, Test Accuracy: 0.8214


C:\Users\Prachi\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


CNN model saved successfully at: models/cnn_model.h5
Training VGG16 model...
Epoch 1/5
106/106 [==============================] - 6968s 66s/step - loss: 0.4350 - accuracy: 0.8009 - val_loss: 0.3491 - val_accuracy: 0.8551
Epoch 2/5
106/106 [==============================] - 9443s 90s/step - loss: 0.2817 - accuracy: 0.8912 - val_loss: 0.2796 - val_accuracy: 0.8883
Epoch 3/5
106/106 [==============================] - 5766s 55s/step - loss: 0.2278 - accuracy: 0.9168 - val_loss: 0.2462 - val_accuracy: 0.9026
Epoch 4/5
106/106 [==============================] - 5631s 53s/step - loss: 0.1972 - accuracy: 0.9290 - val_loss: 0.2225 - val_accuracy: 0.9124
Epoch 5/5
106/106 [==============================] - 5573s 53s/step - loss: 0.1750 - accuracy: 0.9382 - val_loss: 0.2081 - val_accuracy: 0.9193
VGG16 - Test Loss: 0.1643, Test Accuracy: 0.9422
VGG16 model saved successfully at: models/vgg16_model.h5
Training VGG19 model...
Epoch 1/5
106/106 [==============================] - 7083s 67s/step - los

In [ ]:
# 4. Plot Loss and Accuracy Graphs
def plot_metrics(history_dict):
    plt.figure(figsize=(12, 8))
    for model_name, history in history_dict.items():
        plt.plot(history.history['accuracy'], label=f'{model_name} Train Accuracy')
        plt.plot(history.history['val_accuracy'], label=f'{model_name} Val Accuracy')
    plt.title('Model Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()
    plt.show()

    plt.figure(figsize=(12, 8))
    for model_name, history in history_dict.items():
        plt.plot(history.history['loss'], label=f'{model_name} Train Loss')
        plt.plot(history.history['val_loss'], label=f'{model_name} Val Loss')
    plt.title('Model Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

# Plot metrics
plot_metrics(history_dict)

In [ ]:
# Create a folder to save the plots if it does not exist
os.makedirs("plots", exist_ok=True)

# Function to plot individual accuracy and loss graphs for each model
def plot_individual_metrics(history_dict):
    for model_name, history in history_dict.items():
        # Plot Accuracy
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['accuracy'], label='Train Accuracy', color='blue')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
        plt.title(f'{model_name} Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid()
        plt.savefig(f"plots/{model_name.lower()}_accuracy.png", dpi=300)
        plt.show()

        # Plot Loss
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['loss'], label='Train Loss', color='blue')
        plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
        plt.title(f'{model_name} Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid()
        plt.savefig(f"plots/{model_name.lower()}_loss.png", dpi=300)
        plt.show()

# Plot individual metrics for each model
plot_individual_metrics(history_dict)

In [ ]:
# 5. Confusion Matrices and Performance Metrics
for model_name, model in models.items():
    print(f"\nGenerating Confusion Matrix for {model_name}...")
    y_pred = model.predict(test_gen)
    y_pred = np.round(y_pred).astype(int)
    y_true = test_gen.classes

    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    print(f"\nClassification Report for {model_name}:")
    print(classification_report(y_true, y_pred, target_names=['No Tumor', 'Tumor']))